Instalando Pacotes

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas

Fazendo a Raspagem dos dados

In [ ]:
# Top 250 filmes Imdb
import pandas as pd # pacote para manipulação de dados
from bs4 import BeautifulSoup # pacote para raspagem de dados
import requests # pacote para fazer pedido de entrada no site
import re # pacote para expressão regular

url = "https://www.imdb.com/"

# Precisamos desse Headers para que o site aceita a solicitação
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# Solicitando para obter o conteúdo da página
response = requests.get(url, headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

top250 = soup.find('ul', class_='ipc-list navlinkcat__list ipc-list--baseAlt')
top250 = top250.select('a')[1]
top250 = "https://www.imdb.com/" + top250.get('href')

# Agora podemos acessar o link que geramos acima da mesma forma que acessamos o outro:
response = requests.get(top250, headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# Quero selecionar todos os nomes dos filmes
top250_movies = [h3.text for h3 in soup.select('ul h3')]

# expressão regular para remover os números e pontos no início dos títulos
top250_movies = [re.sub(r'^\d+\.\s*', '', movie) for movie in top250_movies] # note que no python não precisamos escapar a \

df = pd.DataFrame({'Nome do Filme': top250_movies})
# aqui ja teriamos os 250 filmes, mas vamos pegar mais informações

# extração do ano, duração, idade permitida
metadata = [item.text for item in soup.select('ul .cli-title-metadata-item')] # da pra selecionar a classe assim tambem

# avaliações
rating = [item.text for item in soup.select('ul .ratingGroup--imdb-rating')]
rating = [re.sub(r'\([^)]*\)', '', r) for r in rating]

# colocando todas as informações no dataframe
df = pd.DataFrame({'Filme': top250_movies, 'Avaliação': rating})
df['Ano'] = metadata[::3]
df['Duracao'] = metadata[1::3]
df['IdadePermitida'] = metadata[2::3]

# Conversor de duração para minutos
def duration_to_minutes(duration):
    match = re.search(r'(\d+)h?', duration)
    hours = int(match.group(1)) if match else 0
    match = re.search(r'(\d+)min', duration)
    minutes = int(match.group(1)) if match else 0
    return hours * 60 + minutes

df['Duracao'] = df['Duracao'].apply(duration_to_minutes)

print(df)